# 0-载入包

In [3]:
#注意添加注释
#2024-10-14：#这个是各个银行的效率比率。之前（7-CNRDS.ipynb）是 各个银行的分支机构。
#代码和分支机构相同，从分支机构直接复制 修改
#2024-10-09：从此程序开始，对每一个程序，设立两个文档：比如7-CNRDS.ipynb是实际操作记录。7-CNRDS-exe.ipynb最后执行文档

In [4]:
library(tidyverse)
library(data.table)
library(readxl)

In [5]:
#library(microbenchmark)

# 8-分支机构数据

## 8.1-读入.csv文件

In [6]:
# Create a son function to read a single CSV file
# .csv文件的第一行是变量名称，第二行是变量说明，第三行开始才是正式的数据
read_csv_file0 <- function(file_name) {
  # Read the header separately
  header <- read_csv(file_name, n_max = 1, col_names = FALSE)
  
  # Read the data, skipping the first two rows
  df <- read_csv(file_name, 
                 col_names = FALSE,
                 skip = 2)
  
  # Set the column names from the header
  colnames(df) <- as.character(header[1,])
  
  return(df)
}

In [7]:
# Set the base path and file details
#银行资产负债表 银行利润表
#国泰安 文件名格式：D:/IDMhs/收益预测-月个股回报率文件-2024-09-23/SRFR_Amnthlyr.csv
#CEIC文件名格式： D:/jianguo/myR/pop-人口数-CEIC-2023-04-21.csv
#CNRDS文件名格式：D:/IDMhs/CNRDS-银行资产负债表-2024-10-12/银行资产负债表.csv
#D:/IDMhs/CNRDS-银行利润表-2024-10-12/银行利润表.csv
#D:/IDMhs/CNRDS-银行概况-2024-10-12/银行概况.csv

#Bank Balance Sheet, Bank Income Statement
dir_base="D:/IDMhs"  #基本文件夹  #下面使用 file.path 函数，所以不用在文件夹之间添加斜杠 "/" 
dir_prefix= "CNRDS"  #子文件夹中的前缀 主干 后缀 
dir_suffix="2024-10-12"    #suffix-后缀。prefix-前缀
file_root <- c("银行概况", "银行资产负债表", "银行利润表")

In [8]:
#基本文件夹 + 子文件夹 + 文件名 
file_name <- file.path(dir_base, 
                       paste0(dir_prefix, "-", file_root, "-", dir_suffix), 
                       paste0(file_root, ".csv"))

In [9]:
print(file_name)

[1] "D:/IDMhs/CNRDS-银行概况-2024-10-12/银行概况.csv"            
[2] "D:/IDMhs/CNRDS-银行资产负债表-2024-10-12/银行资产负债表.csv"
[3] "D:/IDMhs/CNRDS-银行利润表-2024-10-12/银行利润表.csv"        


In [10]:
# Read the CSV files
# 此处读入代码 比较 简单。不像 最初 在 4-finstr中，claude将其读入列表之中，进行各种错误审查

df_names <- c("bank_overview", "bank_balance", "bank_income")
for (i in 1:length(file_name)) {
  assign(df_names[i], read_csv_file0(file_name[i]) ) 
}

Rows: 1 Columns: 11
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (11): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 3629 Columns: 11
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): X2, X4, X6, X7, X8, X9, X10, X11
dbl  (2): X1, X5
date (1): X3

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1 Columns: 28
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (28): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE`

In [11]:
# Count records for each Bank Code in Balance Sheet and Income Statement
balance_count <- bank_balance %>%
  group_by(BankCode) %>%
  summarise(Balance_Sheet_Records = n())

income_count <- bank_income %>%
  group_by(BankCode) %>%
  summarise(Income_Statement_Records = n())

In [12]:
# Merge the counts with the Bank Overview data
result <- bank_overview %>%
  left_join(balance_count, by = "BankCode") %>%
  left_join(income_count, by = "BankCode") %>%
  select(BankCode, BankName, Balance_Sheet_Records, Income_Statement_Records)

In [13]:
# Replace NA with 0 for banks without records in Balance Sheet or Income Statement
result <- result %>%
  mutate(
    Balance_Sheet_Records = replace_na(Balance_Sheet_Records, 0),
    Income_Statement_Records = replace_na(Income_Statement_Records, 0)
  )

In [14]:
# Display the results
print(head(result))

# A tibble: 6 × 4
  BankCode BankName                Balance_Sheet_Records Income_Statement_Reco…¹
     <dbl> <chr>                                   <int>                   <int>
1   100001 安徽青阳农村商业银行股…                     7                       7
2   100002 安徽桐城农村商业银行股…                    16                      16
3   100003 包商银行股份有限公司                       10                      10
4   100004 保定银行股份有限公司                       12                      12
5   100005 北京农村商业银行股份有…                    17                      17
6   100006 北京银行股份有限公司                       17                      17
# ℹ abbreviated name: ¹​Income_Statement_Records


In [15]:
# Summary statistics
summary_stats <- result %>%
  summarise(
    Total_Banks = n(),
    Banks_with_Balance_Sheet = sum(Balance_Sheet_Records > 0),
    Banks_with_Income_Statement = sum(Income_Statement_Records > 0),
    Avg_Balance_Sheet_Records = mean(Balance_Sheet_Records),
    Avg_Income_Statement_Records = mean(Income_Statement_Records),
    Max_Balance_Sheet_Records = max(Balance_Sheet_Records),
    Max_Income_Statement_Records = max(Income_Statement_Records)
  )

In [18]:
print(summary_stats)

# A tibble: 1 × 7
  Total_Banks Banks_with_Balance_Sheet Banks_with_Income_Statement
        <int>                    <int>                       <int>
1        3629                     1093                        1213
# ℹ 4 more variables: Avg_Balance_Sheet_Records <dbl>,
#   Avg_Income_Statement_Records <dbl>, Max_Balance_Sheet_Records <int>,
#   Max_Income_Statement_Records <int>


In [22]:
#记录最多的银行。如下所示是 广东顺德农村商业银行。
#从2004年开始有记录，但是，实际数据从2009年开始
# First, get the Max_Balance_Sheet_Records value
max_records <- summary_stats$Max_Balance_Sheet_Records

# Then, filter the 'result' data frame
max_records_rows <- result[result$Balance_Sheet_Records == max_records, ]

# View the filtered results
print(max_records_rows)

# A tibble: 1 × 4
  BankCode BankName                Balance_Sheet_Records Income_Statement_Reco…¹
     <dbl> <chr>                                   <int>                   <int>
1   100038 广东顺德农村商业银行股…                    20                      20
# ℹ abbreviated name: ¹​Income_Statement_Records


In [17]:
str(summary_stats)

tibble [1 × 7] (S3: tbl_df/tbl/data.frame)
 $ Total_Banks                 : int 3629
 $ Banks_with_Balance_Sheet    : int 1093
 $ Banks_with_Income_Statement : int 1213
 $ Avg_Balance_Sheet_Records   : num 2.05
 $ Avg_Income_Statement_Records: num 2.13
 $ Max_Balance_Sheet_Records   : int 20
 $ Max_Income_Statement_Records: int 20


In [24]:
# Then, filter the 'result' data frame
max_records <- bank_balance[bank_balance$BankCode == "100038", ]

# View the filtered results
#print(max_records)
# View the filtered results
print(head(max_records))

# A tibble: 6 × 28
  BankCode BankName    Date           CBID     IPD   LNOBK   HLTFA DEFIA  ACITRE
     <dbl> <chr>       <date>        <dbl>   <dbl>   <dbl>   <dbl> <dbl>   <dbl>
1   100038 广东顺德农… 2004-12-31  4.80e 9 1.85e10 NA      NA         NA NA     
2   100038 广东顺德农… 2005-12-31  4.13e 9 2.45e10 NA      NA         NA NA     
3   100038 广东顺德农… 2006-12-31  4.97e 9 2.56e10 NA      NA         NA NA     
4   100038 广东顺德农… 2007-12-31  8.41e 9 1.68e 9 NA      NA         NA NA     
5   100038 广东顺德农… 2008-12-31  9.91e 9 4.17e 9 NA      NA         NA NA     
6   100038 广东顺德农… 2009-12-31  1.06e10 5.64e 9  9.27e7  3.22e8     0  4.59e8
# ℹ 19 more variables: LNADV <dbl>, AFSFIA <dbl>, FXA <dbl>, TA <dbl>,
#   BOFIF <dbl>, BWFCBK <dbl>, LNFOBK <dbl>, TKDEP <dbl>, TP <dbl>, INTP <dbl>,
#   LB <dbl>, PIC <dbl>, APIC <dbl>, SR <dbl>, UDP <dbl>, GRKPP <dbl>,
#   EAPC <dbl>, EQU <dbl>, TLE <dbl>


In [25]:
# Step 1: Filter the banks based on bank type
filtered_banks <- bank_overview %>%
  filter(BkNtr %in% c("大型商业银行", "股份制商业银行"))

In [29]:
print(filtered_banks)

# A tibble: 18 × 11
   BankCode BankName  EstbDate   BkNtr  RgsCapt ListOrN Province RgsAdrs BusnScp
      <dbl> <chr>     <date>     <chr>    <dbl> <chr>   <chr>    <chr>   <chr>  
 1   100008 渤海银行… 2005-12-30 股份…  1.78e10 是      天津     天津市… 全国性…
 2   100039 广发银行… 1988-07-08 股份…  2.18e10 否      广东省   广东省… 吸收公…
 3   100053 恒丰银行… 1987-11-23 股份…  1.11e11 否      山东省   山东省… 吸收人…
 4   100059 华夏银行… 1992-10-14 股份…  1.59e10 是      北京     北京市… (一)吸…
 5   100087 交通银行… 1987-03-30 大型…  7.43e10 是      上海     上海市… (一)吸…
 6   100118 平安银行… 1987-12-22 股份…  1.94e10 是      广东省   广东省… (一)吸…
 7   100134 上海浦东… 1992-10-19 股份…  2.94e10 是      上海     上海市… (一)吸…
 8   100155 兴业银行… 1988-08-22 股份…  2.08e10 是      福建省   福建省… 吸收公…
 9   100165 招商银行… 1987-03-31 股份…  2.52e10 是      广东省   广东省… 吸收公…
10   100176 浙商银行… 1993-04-16 股份…  2.75e10 是      浙江省   浙江省… 吸收公…
11   100178 中国工商… 1985-11-22 大型…  3.56e11 是      北京     北京市… 办理人…
12   100179 中国光大… 1992-06-18 股份…  5.91e10 是      北京     北京市… 吸收公…
13   100180 中国建设… 2004-09-17

In [30]:
print(filtered_banks[,1:2])

# A tibble: 18 × 2
   BankCode BankName                    
      <dbl> <chr>                       
 1   100008 渤海银行股份有限公司        
 2   100039 广发银行股份有限公司        
 3   100053 恒丰银行股份有限公司        
 4   100059 华夏银行股份有限公司        
 5   100087 交通银行股份有限公司        
 6   100118 平安银行股份有限公司        
 7   100134 上海浦东发展银行股份有限公司
 8   100155 兴业银行股份有限公司        
 9   100165 招商银行股份有限公司        
10   100176 浙商银行股份有限公司        
11   100178 中国工商银行股份有限公司    
12   100179 中国光大银行股份有限公司    
13   100180 中国建设银行股份有限公司    
14   100181 中国民生银行股份有限公司    
15   100182 中国农业银行股份有限公司    
16   100183 中国银行股份有限公司        
17   100184 中国邮政储蓄银行股份有限公司
18   100185 中信银行股份有限公司        


In [27]:
# Step 2: Compile statistics by province
# 在该省 有几个大银行，有几种性质的大银行
province_stats <- filtered_banks %>%
  group_by(Province) %>%
  summarise(
    Count = n(),
    #AvgAssets = mean(TotalAssets, na.rm = TRUE),
    #TotalAssets = sum(TotalAssets, na.rm = TRUE),
    UniqueTypes = n_distinct(BkNtr)
  ) %>%
  arrange(desc(Count))

In [28]:
# Step 3: View the results
print(province_stats)

# A tibble: 7 × 3
  Province Count UniqueTypes
  <chr>    <int>       <int>
1 北京         9           2
2 广东省       3           1
3 上海         2           2
4 天津         1           1
5 山东省       1           1
6 浙江省       1           1
7 福建省       1           1


# 下面是之前7-CNRDS的代码

In [ ]:
#下面是之前7-CNRDS的代码

In [ ]:
# Create a function to read a single CSV file
read_csv_file <- function(file_number) {
  file_na <- paste0(base_path, file_prefix, file_number, file_extension)
  df=read_csv_file0(file_na)
  return(df)
}

In [ ]:
# Read all CSV files and combine them into a single data frame
Ins_entry <- map_df(number_range, read_csv_file)

Rows: 1 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (7): X1, X2, X3, X4, X5, X6, X7

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 100000 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (5): X1, X2, X3, X4, X5
date (2): X6, X7

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (7): X1, X2, X3, X4, X5, X6, X7

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 100000 Columns: 7
── Column specification ───────────────────

In [ ]:
# Print the dimensions of the combined data frame
print(paste("Dimensions of the combined data frame:", 
            nrow(Ins_entry), "rows and", ncol(Ins_entry), "columns"))

[1] "Dimensions of the combined data frame: 279588 rows and 7 columns"


In [ ]:
# Display the first few rows of the combined data frame
print(head(Ins_entry))

# Display the column names
#print(colnames(Ins_entry))

# Display the structure of the combined data frame
#str(Ins_entry)

# A tibble: 6 × 7
  Inscode         Intype     Insname       Insadr  Inscity Issdate    Apprdate  
  <chr>           <chr>      <chr>         <chr>   <chr>   <date>     <date>    
1 A0002K244010001 政策性银行 中国农业发展… 广州市… 广东省… 2024-04-26 1996-10-14
2 A0002L315290001 政策性银行 中国农业发展… 内蒙古… 内蒙古… 2024-01-04 1996-09-26
3 A0002L335060001 政策性银行 中国农业发展… 福建省… 福建省… 2024-06-12 1996-12-17
4 A0002L352270001 政策性银行 中国农业发展… 贵州省… 贵州省… 2024-06-03 1996-12-19
5 A0002L365020001 政策性银行 中国农业发展… 新疆维… 新疆维… 2024-04-07 1998-07-08
6 A0002N365020001 政策性银行 中国农业发展… 新疆维… 新疆维… 2024-04-07 1998-07-08


In [ ]:
Ins_exit <- read_csv_file0(file_name2)

Rows: 1 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (8): X1, X2, X3, X4, X5, X6, X7, X8

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 38735 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (5): X1, X2, X3, X4, X5
date (3): X6, X7, X8

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
# Print the dimensions of the combined data frame
print(paste("Dimensions of the combined data frame:", 
            nrow(Ins_exit), "rows and", ncol(Ins_exit), "columns"))

[1] "Dimensions of the combined data frame: 38735 rows and 8 columns"


In [ ]:
print(head(Ins_exit))

# A tibble: 6 × 8
  Inscode         Intype Insname Insadr Inscity Issdate    Apprdate   Exitdt    
  <chr>           <chr>  <chr>   <chr>  <chr>   <date>     <date>     <date>    
1 A0002L335090002 政策…  中国农… 福建…  福建省… 2007-06-12 1997-03-28 2010-07-28
2 A0002L337130001 政策…  中国农… 临沂…  山东省… 2007-06-22 1996-11-12 2008-11-03
3 A0002N237020001 政策…  中国农… 山东…  山东省… 2007-05-14 1996-09-26 2008-02-21
4 A0002N314020001 政策…  中国农… 大同…  山西省… 2007-04-25 1997-05-07 2015-07-23
5 A0002N314030001 政策…  中国农… 山西…  山西省… 2007-04-24 1997-05-04 2014-02-13
6 A0002N314040001 政策…  中国农… 山西…  山西省… 2007-04-27 1997-04-30 2015-11-17


## 8.2-合并Ins_2-不合并

In [ ]:
#后来决定不合并。所以全部注释

In [ ]:
# Add Exitdt column to Ins_entry  #给Ins_entry增加一列。便于合并两个数据框
# Ins_entry <- Ins_entry %>%
#   mutate(Exitdt = as.Date("2024-12-31"))

In [ ]:
# print(head(Ins_entry,3))

In [ ]:
# # Merge Ins_entry and Ins_exit
# Ins_2 <- bind_rows(Ins_entry, Ins_exit)

In [ ]:
# dim(Ins_2)

## 8.3-生成新列

### 8.3.1-最早成立年份

In [ ]:
# Print the row before 1980-01-01
cat("Row with Apprdate before 1980-01-01:\n")
print(Ins_entry %>% filter(Apprdate < as.Date("1980-01-01")))

Row with typo before correction:
# A tibble: 10,364 × 7
   Inscode         Intype   Insname         Insadr Inscity Issdate    Apprdate  
   <chr>           <chr>    <chr>           <chr>  <chr>   <date>     <date>    
 1 B0001L243120001 商业银行 中国工商银行股… 湖南…  湖南省… 2007-05-18 1950-01-01
 2 B0001L252010001 商业银行 中国工商银行股… 贵州…  贵州省… 2022-02-28 1972-10-01
 3 B0001L343120001 商业银行 中国工商银行股… 湖南…  湖南省… 2023-05-11 1950-01-01
 4 B0001M212000006 商业银行 中国工商银行股… 天津…  天津市  2022-01-28 1969-01-01
 5 B0001M212000007 商业银行 中国工商银行股… 天津…  天津市  2021-12-02 1973-08-16
 6 B0001S212000019 商业银行 中国工商银行股… 天津…  天津市  2021-12-02 1978-01-01
 7 B0001S212000031 商业银行 中国工商银行股… 天津…  天津市  2022-02-09 1969-01-01
 8 B0001S221020040 商业银行 中国工商银行股… 大连…  辽宁省… 2021-12-24 1954-01-01
 9 B0001S221020090 商业银行 中国工商银行股… 辽宁…  辽宁省… 2021-12-24 1978-03-01
10 B0001S221020097 商业银行 中国工商银行股… 辽宁…  辽宁省… 2021-12-24 1975-12-29
# ℹ 10,354 more rows


In [ ]:
# 不知道是什么原因，存在1万多行，成立日期在1980-01-01之前。乃统计1984年之后的分支情况
start_year <- 1984

In [ ]:
# Create a sequence of years from start_year to 2024
years <- seq(start_year, 2024)

### 8.3.2-na处理

In [ ]:
# Find rows where Apprdate is NA
na_rows_entry <- Ins_entry %>% filter(is.na(Apprdate))
dim(na_rows_entry)

In [ ]:
# Find rows where Apprdate is NA
na_rows_exit <- Ins_exit %>% filter(is.na(Apprdate))
dim(na_rows_exit)

### 8.3.3-生成新列

In [ ]:
#结构代码。机构名称。机构类型。机构类型首字母。省份、城市。
#对于Apprdate=NA的行，令Apprdate=Issdate
process_dataframe <- function(df) {
  df %>%
    mutate(
      Inscode2 = str_sub(Inscode, 1, 5),
      Insname2 = str_sub(Insname, 1, 6),
      Intype2 = as.factor(Intype),
      Intype3 = str_sub(Inscode, 1, 1),
      Ins_prov = str_replace(Inscity, "-.+$", ""),
      Ins_city = str_extract(Inscity, "(?<=-).*") %>% 
                 ifelse(is.na(.), Inscity, .),
      Apprdate = if_else(is.na(Apprdate), Issdate, Apprdate)
    )
}

In [ ]:
# Assuming Ins_2 is your original dataframe
Ins_entry2 <- process_dataframe(Ins_entry)
Ins_exit2 <- process_dataframe(Ins_exit)

In [ ]:
# Find rows where Apprdate is NA
na_rows_entry <- Ins_entry2 %>% filter(is.na(Apprdate))
dim(na_rows_entry)

[1]  0 13

In [ ]:
na_rows_exit <- Ins_exit2 %>% filter(is.na(Apprdate))
dim(na_rows_exit)

[1]  0 14

### 8.3.4-试探-验证首字母

In [ ]:
# Validate the hypothesis  #验证首字母是否对应机构类型  #ctrl+/ 添加注释
#下面是最初的代码。后来转换成 函数 
# validation_result <- Ins_entry2 %>%
#   group_by(Intype3, Intype) %>%
#   summarise(count = n(), .groups = 'drop') %>%
#   arrange(Intype3, desc(count))

# # Print the validation result
# print(validation_result)

In [ ]:
validate_insurance_types <- function(Ins_df) {
  validation_result <- Ins_df %>%
    group_by(Intype3, Intype) %>%
    summarise(count = n(), .groups = 'drop') %>%
    arrange(Intype3, desc(count))
  
  return(validation_result)
}

In [ ]:
validation_result = validate_insurance_types(Ins_entry2)

In [ ]:
# Print the validation result
print(validation_result)

# A tibble: 17 × 3
   Intype3 Intype            count
   <chr>   <chr>             <int>
 1 A       政策性银行           23
 2 B       商业银行         242203
 3 C       农村合作银行       1259
 4 E       农村信用社        28024
 5 F       资金互助社           48
 6 G       开发性金融机构        1
 7 I       邮政储蓄网点         44
 8 J       金融资产管理公司    129
 9 K       信托公司             83
10 L       财务公司            363
11 M       金融租赁公司         81
12 N       汽车金融公司         32
13 P       货币经纪公司          7
14 Q       贷款公司             13
15 S       村镇银行           7198
16 X       消费金融公司         32
17 Z       其他类金融机构       48


In [ ]:
Ins_exit2 <- process_dataframe(Ins_exit)

In [ ]:
str(Ins_exit2)

tibble [38,735 × 14] (S3: tbl_df/tbl/data.frame)
 $ Inscode : chr [1:38735] "A0002L335090002" "A0002L337130001" "A0002N237020001" "A0002N314020001" ...
 $ Intype  : chr [1:38735] "政策性银行" "政策性银行" "政策性银行" "政策性银行" ...
 $ Insname : chr [1:38735] "中国农业发展银行宁德分行营业部" "中国农业发展银行临沂市分行营业部" "中国农业发展银行青岛市分行营业部" "中国农业发展银行大同市分行营业部" ...
 $ Insadr  : chr [1:38735] "福建省宁德市蕉城南路42号" "临沂市金雀山路中段71号" "山东省青岛市东海西路49号丙" "大同市新建北路红卫里6号" ...
 $ Inscity : chr [1:38735] "福建省-宁德市" "山东省-临沂市" "山东省-青岛市" "山西省-大同市" ...
 $ Issdate : Date[1:38735], format: "2007-06-12" "2007-06-22" ...
 $ Apprdate: Date[1:38735], format: "1997-03-28" "1996-11-12" ...
 $ Exitdt  : Date[1:38735], format: "2010-07-28" "2008-11-03" ...
 $ Inscode2: chr [1:38735] "A0002" "A0002" "A0002" "A0002" ...
 $ Insname2: chr [1:38735] "中国农业发展" "中国农业发展" "中国农业发展" "中国农业发展" ...
 $ Intype2 : Factor w/ 13 levels "财务公司","城市信用社",..: 12 12 12 12 12 12 12 12 12 12 ...
 $ Intype3 : chr [1:38735] "A" "A" "A" "A" ...
 $ Ins_prov: chr [1:38735] "福建省" "山东省" "山东省" "山西省" ..

In [ ]:
validation_result2 = validate_insurance_types(Ins_exit2)

In [ ]:
# Print the validation result
print(validation_result2)

# A tibble: 13 × 3
   Intype3 Intype       count
   <chr>   <chr>        <int>
 1 A       政策性银行      96
 2 B       商业银行     26260
 3 C       农村合作银行   500
 4 D       城市信用社      93
 5 E       农村信用社   11038
 6 F       资金互助社      20
 7 I       邮政储蓄网点   365
 8 K       信托公司         2
 9 L       财务公司        34
10 M       金融租赁公司     2
11 Q       贷款公司        13
12 S       村镇银行       311
13 Z       其他金融机构     1


### 8.3.5-生成df3

In [ ]:
Ins_entry3 <- Ins_entry2 %>% filter(Intype3 %in% c("B", "C", "D", "E", "S"))

In [ ]:
Ins_exit3 <- Ins_exit2 %>% filter(Intype3 %in% c("B", "C", "D", "E", "S"))

In [ ]:
str(Ins_exit3)

tibble [38,202 × 14] (S3: tbl_df/tbl/data.frame)
 $ Inscode : chr [1:38202] "B0001G211000002" "B0001G232010001" "B0001G237010001" "B0001K231000002" ...
 $ Intype  : chr [1:38202] "商业银行" "商业银行" "商业银行" "商业银行" ...
 $ Insname : chr [1:38202] "中国工商银行股份有限公司私人银行部北京分部" "中国工商银行股份有限公司牡丹卡中心南京分中心" "中国工商银行股份有限公司私人银行部济南分部" "中国工商银行股份有限公司私人银行部上海分部" ...
 $ Insadr  : chr [1:38202] "北京市朝阳区建国门外大街1号院1号楼国贸大厦28层" "南京市汉中路95号" "济南市经四路310号银工大厦东副楼二楼" "上海市浦明路132号" ...
 $ Inscity : chr [1:38202] "北京市" "江苏省-南京市" "山东省-济南市" "上海市" ...
 $ Issdate : Date[1:38202], format: "2011-05-05" "2013-03-26" ...
 $ Apprdate: Date[1:38202], format: "2008-10-17" "2002-12-18" ...
 $ Exitdt  : Date[1:38202], format: "2014-08-15" "2022-05-13" ...
 $ Inscode2: chr [1:38202] "B0001" "B0001" "B0001" "B0001" ...
 $ Insname2: chr [1:38202] "中国工商银行" "中国工商银行" "中国工商银行" "中国工商银行" ...
 $ Intype2 : Factor w/ 13 levels "财务公司","城市信用社",..: 9 9 9 9 9 9 9 9 9 9 ...
 $ Intype3 : chr [1:38202] "B" "B" "B" "B" ...
 $ Ins_prov: chr [1:38202] "北京市" "江苏省" "山

In [ ]:
#注意：因为中间版本调整，所以下面的Ins_entry, Ins_entry2, Ins_entry3名称需要注意。
#现在基本对应最初的数据框，经过处理的数据框（比如添加省份），选取银行类型的数据框。

### 8.3.6-运行速度比较-注释

In [ ]:
#以下内容 可以先 注释

In [ ]:
# #crossing-uncount速度对比
# microbenchmark(
#   code1 = {
#     years = 1984:2024
#     Ins_4 <- Ins_entry %>% crossing(year = years)
#   },
#   code2 = {
#     Ins_4b <- Ins_entry %>% 
#       mutate(freq = 2024-1984+1) %>% 
#       uncount(freq) %>%
#       group_by(Inscode) %>%
#       mutate(year = 1984:2024) %>%
#       ungroup()
#   },
#   times = 10
# )

expr,time
<fct>,<dbl>
code2,1677887500
code1,1549900100
code2,2288300700
code2,2367746700
code1,1478808300
code2,2387621800
code2,2461381600
code1,1581086600
code1,2430333100


In [ ]:
# #crossing-uncount速度对比
# microbenchmark(
#   code1 = {
#     years = 1984:2024
#     Ins_4 <- Ins_entry %>% crossing(year = years)
#   },
#   code2 = {
#     Ins_4b <- Ins_entry %>% 
#       mutate(freq = 2024-1984+1) %>% 
#       uncount(freq) %>%
#       group_by(Inscode) %>%
#       mutate(year = 1984:2024) %>%
#       ungroup()
#   },
#   code3 = {
#   Ins_4c <- setDT(Ins_entry)[, .(year = 1984:2024), by = .(Inscode)]
# },
#   times = 10
# )

In [ ]:
# microbenchmark(
#   code1 = {
#     years = 1984:2024
#     Ins_4 <- Ins_entry %>% crossing(year = years)
#   },
#   code2 = {
#     Ins_4b <- Ins_entry %>% 
#       mutate(freq = 2024-1984+1) %>% 
#       uncount(freq) %>%
#       group_by(Inscode) %>%
#       mutate(year = 1984:2024) %>%
#       ungroup()
#   },
#   code3 = {
#     Ins_4c <- setDT(Ins_entry)[, .(year = 1984:2024), by = .(Inscode)]
#   },
#   times = 10
# )

expr,time
<fct>,<dbl>
code2,1465660500
code3,565325300
code1,2269470900
code2,1563565400
code1,1671480400
code2,2045950700
code3,1170355500
code2,1736629000
code2,2065278900


In [ ]:
# # Run the microbenchmark
# benchmark_results <- microbenchmark(
#   code1 = {
#     years = 1984:2024
#     Ins_4 <- Ins_entry %>% crossing(year = years)
#   },
#   code2 = {
#     Ins_4b <- Ins_entry %>% 
#       mutate(freq = 2024-1984+1) %>% 
#       uncount(freq) %>%
#       group_by(Inscode) %>%
#       mutate(year = 1984:2024) %>%
#       ungroup()
#   },
#   code3 = {
#     Ins_4c <- setDT(Ins_entry)[, .(year = 1984:2024), by = .(Inscode)]
#   },
#   times = 10
# )



ERROR: [1m[33mError[39m in `select()`:[22m
[33m![39m Can't subset columns that don't exist.
[31m✖[39m Column `sd` doesn't exist.


In [ ]:
# # Process the results
# summary_results <- as.data.frame(benchmark_results) %>%
#   group_by(expr) %>%
#   summarise(
#     mean = mean(time),
#     sd = sd(time),
#     .groups = 'drop'
#   ) %>%
#   mutate(
#     mean_seconds = mean / 1e9,
#     sd_seconds = sd / 1e9,
#     mean_ms = mean / 1e6,
#     sd_ms = sd / 1e6
#   )

# # Print the results
# print(summary_results, digits = 4)

# # Create a more readable output
# cat("\nReadable summary:\n")
# for (i in 1:nrow(summary_results)) {
#   cat(sprintf("%s:\n", summary_results$expr[i]))
#   cat(sprintf("  Mean time: %.4f seconds (%.2f ms)\n", 
#               summary_results$mean_seconds[i], 
#               summary_results$mean_ms[i]))
#   cat(sprintf("  Std Dev:   %.4f seconds (%.2f ms)\n\n", 
#               summary_results$sd_seconds[i], 
#               summary_results$sd_ms[i]))
# }

# A tibble: 3 × 7
  expr        mean         sd mean_seconds sd_seconds mean_ms sd_ms
  <fct>      <dbl>      <dbl>        <dbl>      <dbl>   <dbl> <dbl>
1 code1 1699287980 322115773.        1.70       0.322   1699.  322.
2 code2 2255888290 315646943.        2.26       0.316   2256.  316.
3 code3  585051540 100012681.        0.585      0.100    585.  100.

Readable summary:
code1:
  Mean time: 1.6993 seconds (1699.29 ms)
  Std Dev:   0.3221 seconds (322.12 ms)

code2:
  Mean time: 2.2559 seconds (2255.89 ms)
  Std Dev:   0.3156 seconds (315.65 ms)

code3:
  Mean time: 0.5851 seconds (585.05 ms)
  Std Dev:   0.1000 seconds (100.01 ms)



In [ ]:
# Check for any unexpected values in Intype3
# unexpected_types <- Ins_3 %>%
#   filter(!Intype3 %in% c("B", "C", "D", "E", "S"))

# if (nrow(unexpected_types) > 0) {
#   cat("Warning: Unexpected values found in Intype3\n")
#   print(table(unexpected_types$Intype3))
# } else {
#   cat("No unexpected values found in Intype3\n")
# }

## 8.4-计算各省分支

In [ ]:
###前面是试探代码。直接计入8.4.1

In [ ]:
#第一部分使用data.table。后来发现tidyverse速度可能还要稍快一些。同时语法更加熟悉。所以注释掉

In [ ]:
# # Convert Ins_entry2 to a data.table if it's not already
# setDT(Ins_entry2)

In [ ]:
# # Calculate the count for each province and year
# summary_stats <- Ins_entry2[, .(
#   branch_count = sapply(years, function(y) sum(year(Apprdate) <= y))
# ), by = Ins_prov][, .(Ins_prov, year = rep(years, .N), branch_count = unlist(branch_count))]

In [ ]:
# result <- Ins_entry2[, .(year = years, 
#                         count = as.integer(year >= year(Apprdate))), 
#                     by = .(Inscode, Ins_prov)]

ERROR: Error in year >= year(Apprdate): comparison (>=) is possible only for atomic and list types


In [ ]:
# result <- Ins_entry2[, {
#   .(year = years,
#     count = as.integer(years >= year(Apprdate)))
# }, by = .(Inscode, Ins_prov)]

In [ ]:
# summary_stats2 <- Ins_entry2[, {
#   # Count branches for each year
#   yearly_counts <- sapply(years, function(y) sum(year(Apprdate) <= y))
  
#   # Create a data.table with years and counts
#   .(year = years, branch_count = yearly_counts)
# }, by = Ins_prov]

In [ ]:
# print(head(summary_stats))

   Ins_prov year branch_count
1:   北京市 1984           15
2:   北京市 1985          306
3:   北京市 1986          324
4:   北京市 1987          391
5:   北京市 1988          454
6:   北京市 1989          550


In [ ]:
# print(head(summary_stats2))

   Ins_prov year branch_count
1:   北京市 1984           15
2:   北京市 1985          306
3:   北京市 1986          324
4:   北京市 1987          391
5:   北京市 1988          454
6:   北京市 1989          550


In [ ]:
# summary_stats3 <- Ins_entry2 %>%
#   # Group by province
#   group_by(Ins_prov) %>%
#   # Calculate cumulative counts for each year
#   summarise(
#     branch_count = list(map_int(years, ~sum(year(Apprdate) <= .x)))
#   ) %>%
#   # Unnest the results
#   unnest(branch_count) %>%
#   # Add the year column
#   mutate(year = rep(years, n_distinct(Ins_prov))) %>%
#   # Reorder columns to match the data.table output
#   select(Ins_prov, year, branch_count)



In [ ]:
# summary_stats4 <- Ins_entry2 %>%
#   group_by(Ins_prov) %>%
#   summarise(
#     tibble(
#       year = years,
#       branch_count = map_int(years, ~sum(year(Apprdate) <= .x))
#     )
#   ) %>%
#   ungroup()

Warning message:
"Returning more (or less) than 1 row per `summarise()` group was deprecated in
dplyr 1.1.0.
ℹ Please use `reframe()` instead.
ℹ When switching from `summarise()` to `reframe()`, remember that `reframe()`
  always returns an ungrouped data frame and adjust accordingly."
`summarise()` has grouped output by 'Ins_prov'. You can override using the
`.groups` argument.


In [ ]:
# summary_stats5 <- Ins_entry2 %>%
#   group_by(Ins_prov) %>%
#   reframe(
#     tibble(
#       year = years,
#       branch_count = map_int(years, ~sum(year(Apprdate) <= .x))
#     )
#   )

In [ ]:
# print(head(summary_stats5))

# A tibble: 6 × 3
  Ins_prov  year branch_count
  <chr>    <int>        <int>
1 上海市    1984          389
2 上海市    1985          390
3 上海市    1986          392
4 上海市    1987          435
5 上海市    1988          536
6 上海市    1989          607


In [ ]:
# # Sort both results by Ins_prov and year
# summary_stats2_sorted <- summary_stats2[order(Ins_prov, year)]
# summary_stats5_sorted <- summary_stats5 %>% arrange(Ins_prov, year)

# # Now let's compare them
# are_equal <- all.equal(
#   as.data.frame(summary_stats2_sorted), 
#   as.data.frame(summary_stats5_sorted)
# )

# print(are_equal)

[1] TRUE


In [ ]:
# summary_stats6 <- Ins_entry2 %>%
#   group_by(Ins_prov) %>%
#   reframe(
#     tibble(
#       year = years,
#       branch_count = map_int(years, ~sum(year(Apprdate) <= .))
#     )
#   )

In [ ]:
# summary_stats7 <- Ins_entry2 %>%
#   group_by(Ins_prov) %>%
#   reframe(
#     year = years,
#     branch_count = map_int(years, ~sum(year(Apprdate) <= .)
#     )
#   )

In [ ]:
# all.equal(summary_stats5, summary_stats6)

[1] TRUE

In [ ]:
# all.equal(summary_stats5, summary_stats7)

[1] TRUE

### 8.4.1-各省分支

In [ ]:
#最早的计算比较仔细。考虑成立和退出的年份
#但是一则对总体影响不大，一则为了加快代码。暂时注释掉

In [ ]:
# fraction_of_year <- function(appr_date, exit_date, year) {
#   if (year == year(appr_date) || year == year(exit_date)) {
#     year_start <- as.Date(paste0(year, "-01-01"))
#     year_end <- as.Date(paste0(year, "-12-31"))
    
#     actual_start <- max(appr_date, year_start)
#     actual_end <- min(exit_date, year_end)
    
#     days_in_year <- if(leap_year(year)) 366 else 365
    
#     fraction <- as.numeric(actual_end - actual_start + 1) / days_in_year
#   } else {
#     fraction <- 1
#   }

#   return(fraction)
# }

In [ ]:
Ins_entry4 <- Ins_entry3 %>%
  group_by(Ins_prov) %>%
  reframe(
    tibble(
      year = years,
      branch_count = map_int(years, ~sum(year(Apprdate) <= .))
    )
  )

In [ ]:
str(Ins_exit3)

tibble [38,202 × 14] (S3: tbl_df/tbl/data.frame)
 $ Inscode : chr [1:38202] "B0001G211000002" "B0001G232010001" "B0001G237010001" "B0001K231000002" ...
 $ Intype  : chr [1:38202] "商业银行" "商业银行" "商业银行" "商业银行" ...
 $ Insname : chr [1:38202] "中国工商银行股份有限公司私人银行部北京分部" "中国工商银行股份有限公司牡丹卡中心南京分中心" "中国工商银行股份有限公司私人银行部济南分部" "中国工商银行股份有限公司私人银行部上海分部" ...
 $ Insadr  : chr [1:38202] "北京市朝阳区建国门外大街1号院1号楼国贸大厦28层" "南京市汉中路95号" "济南市经四路310号银工大厦东副楼二楼" "上海市浦明路132号" ...
 $ Inscity : chr [1:38202] "北京市" "江苏省-南京市" "山东省-济南市" "上海市" ...
 $ Issdate : Date[1:38202], format: "2011-05-05" "2013-03-26" ...
 $ Apprdate: Date[1:38202], format: "2008-10-17" "2002-12-18" ...
 $ Exitdt  : Date[1:38202], format: "2014-08-15" "2022-05-13" ...
 $ Inscode2: chr [1:38202] "B0001" "B0001" "B0001" "B0001" ...
 $ Insname2: chr [1:38202] "中国工商银行" "中国工商银行" "中国工商银行" "中国工商银行" ...
 $ Intype2 : Factor w/ 13 levels "财务公司","城市信用社",..: 9 9 9 9 9 9 9 9 9 9 ...
 $ Intype3 : chr [1:38202] "B" "B" "B" "B" ...
 $ Ins_prov: chr [1:38202] "北京市" "江苏省" "山

In [ ]:
print(head(Ins_entry4))

# A tibble: 6 × 3
  Ins_prov  year branch_count
  <chr>    <int>        <int>
1 上海市    1984          389
2 上海市    1985          390
3 上海市    1986          392
4 上海市    1987          435
5 上海市    1988          536
6 上海市    1989          607


In [ ]:
Ins_exit3b <- as.data.table(Ins_exit3)

In [ ]:
str(Ins_exit3b)

Classes 'data.table' and 'data.frame':	38202 obs. of  14 variables:
 $ Inscode : chr  "B0001G211000002" "B0001G232010001" "B0001G237010001" "B0001K231000002" ...
 $ Intype  : chr  "商业银行" "商业银行" "商业银行" "商业银行" ...
 $ Insname : chr  "中国工商银行股份有限公司私人银行部北京分部" "中国工商银行股份有限公司牡丹卡中心南京分中心" "中国工商银行股份有限公司私人银行部济南分部" "中国工商银行股份有限公司私人银行部上海分部" ...
 $ Insadr  : chr  "北京市朝阳区建国门外大街1号院1号楼国贸大厦28层" "南京市汉中路95号" "济南市经四路310号银工大厦东副楼二楼" "上海市浦明路132号" ...
 $ Inscity : chr  "北京市" "江苏省-南京市" "山东省-济南市" "上海市" ...
 $ Issdate : Date, format: "2011-05-05" "2013-03-26" ...
 $ Apprdate: Date, format: "2008-10-17" "2002-12-18" ...
 $ Exitdt  : Date, format: "2014-08-15" "2022-05-13" ...
 $ Inscode2: chr  "B0001" "B0001" "B0001" "B0001" ...
 $ Insname2: chr  "中国工商银行" "中国工商银行" "中国工商银行" "中国工商银行" ...
 $ Intype2 : Factor w/ 13 levels "财务公司","城市信用社",..: 9 9 9 9 9 9 9 9 9 9 ...
 $ Intype3 : chr  "B" "B" "B" "B" ...
 $ Ins_prov: chr  "北京市" "江苏省" "山东省" "上海市" ...
 $ Ins_city: chr  "北京市" "南京市" "济南市" "上海市" ...
 - attr(*, ".internal.selfref

In [ ]:
Ins_entry4b <- Ins_entry3b[, {
  # Count branches for each year
  yearly_counts <- sapply(years, function(y) sum(year(Apprdate) <= y))
  
  # Create a data.table with years and counts
  .(year = years, branch_count = yearly_counts)
}, by = Ins_prov]

### 8.4.2-比较tb和dt

In [ ]:
#比较一下tb格式和dt格式数据框的速度是否有差异。似乎速度没有什么差异

In [ ]:
#不能使用这个。会将两个数据框都变成data.table。不是真复制。可以使用最下面两行
# Ins_exit3b=Ins_exit3
# setDT(Ins_exit3b)

# # Ensure Ins_exit3 is a tibble
# Ins_exit3 <- as_tibble(Ins_exit3)

# # Create a data.table version
# Ins_exit3_dt <- as.data.table(Ins_exit3)

In [ ]:
str(Ins_exit3)

tibble [38,202 × 14] (S3: tbl_df/tbl/data.frame)
 $ Inscode : chr [1:38202] "B0001G211000002" "B0001G232010001" "B0001G237010001" "B0001K231000002" ...
 $ Intype  : chr [1:38202] "商业银行" "商业银行" "商业银行" "商业银行" ...
 $ Insname : chr [1:38202] "中国工商银行股份有限公司私人银行部北京分部" "中国工商银行股份有限公司牡丹卡中心南京分中心" "中国工商银行股份有限公司私人银行部济南分部" "中国工商银行股份有限公司私人银行部上海分部" ...
 $ Insadr  : chr [1:38202] "北京市朝阳区建国门外大街1号院1号楼国贸大厦28层" "南京市汉中路95号" "济南市经四路310号银工大厦东副楼二楼" "上海市浦明路132号" ...
 $ Inscity : chr [1:38202] "北京市" "江苏省-南京市" "山东省-济南市" "上海市" ...
 $ Issdate : Date[1:38202], format: "2011-05-05" "2013-03-26" ...
 $ Apprdate: Date[1:38202], format: "2008-10-17" "2002-12-18" ...
 $ Exitdt  : Date[1:38202], format: "2014-08-15" "2022-05-13" ...
 $ Inscode2: chr [1:38202] "B0001" "B0001" "B0001" "B0001" ...
 $ Insname2: chr [1:38202] "中国工商银行" "中国工商银行" "中国工商银行" "中国工商银行" ...
 $ Intype2 : Factor w/ 13 levels "财务公司","城市信用社",..: 9 9 9 9 9 9 9 9 9 9 ...
 $ Intype3 : chr [1:38202] "B" "B" "B" "B" ...
 $ Ins_prov: chr [1:38202] "北京市" "江苏省" "山

In [ ]:
Ins_exit3b <- as.data.table(Ins_exit3)

In [ ]:
str(Ins_exit3b)

Classes 'data.table' and 'data.frame':	38202 obs. of  14 variables:
 $ Inscode : chr  "B0001G211000002" "B0001G232010001" "B0001G237010001" "B0001K231000002" ...
 $ Intype  : chr  "商业银行" "商业银行" "商业银行" "商业银行" ...
 $ Insname : chr  "中国工商银行股份有限公司私人银行部北京分部" "中国工商银行股份有限公司牡丹卡中心南京分中心" "中国工商银行股份有限公司私人银行部济南分部" "中国工商银行股份有限公司私人银行部上海分部" ...
 $ Insadr  : chr  "北京市朝阳区建国门外大街1号院1号楼国贸大厦28层" "南京市汉中路95号" "济南市经四路310号银工大厦东副楼二楼" "上海市浦明路132号" ...
 $ Inscity : chr  "北京市" "江苏省-南京市" "山东省-济南市" "上海市" ...
 $ Issdate : Date, format: "2011-05-05" "2013-03-26" ...
 $ Apprdate: Date, format: "2008-10-17" "2002-12-18" ...
 $ Exitdt  : Date, format: "2014-08-15" "2022-05-13" ...
 $ Inscode2: chr  "B0001" "B0001" "B0001" "B0001" ...
 $ Insname2: chr  "中国工商银行" "中国工商银行" "中国工商银行" "中国工商银行" ...
 $ Intype2 : Factor w/ 13 levels "财务公司","城市信用社",..: 9 9 9 9 9 9 9 9 9 9 ...
 $ Intype3 : chr  "B" "B" "B" "B" ...
 $ Ins_prov: chr  "北京市" "江苏省" "山东省" "上海市" ...
 $ Ins_city: chr  "北京市" "南京市" "济南市" "上海市" ...
 - attr(*, ".internal.selfref

### 8.4.3-exit

In [ ]:
Ins_exit4 <- Ins_exit3 %>%
  group_by(Ins_prov) %>%
  reframe(
    tibble(
      year = years,
      branch_count = map_int(years, ~sum(year(Apprdate) <= . & . <= year(Exitdt)))
    )
  )

In [ ]:
Ins_exit4b <- Ins_exit3b[, {
  # Count branches for each year
  yearly_counts <- sapply(years, function(y) {
    sum(year(Apprdate) <= y & y <= year(Exitdt))
  })
  
  # Create a data.table with years and counts
  .(year = years, branch_count = yearly_counts)
}, by = Ins_prov]

In [ ]:
print(head(Ins_exit4))

# A tibble: 6 × 3
  Ins_prov  year branch_count
  <chr>    <int>        <int>
1 上海市    1984           41
2 上海市    1985           41
3 上海市    1986           43
4 上海市    1987           43
5 上海市    1988           50
6 上海市    1989           51


In [ ]:
#比较两者结果是否相同
# Sort both results by Ins_prov and year
Ins_exit4b_sorted <- Ins_exit4b[order(Ins_prov, year)]
Ins_exit4_sorted <- Ins_exit4 %>% arrange(Ins_prov, year)

# Now let's compare them
are_equal <- all.equal(
  as.data.frame(Ins_exit4b_sorted), 
  as.data.frame(Ins_exit4_sorted)
)

print(are_equal)

[1] TRUE
